# Platform API - V0.9 B2B Integration Guide

**Last Updated**: 23 FEB 2026
**Version**: 0.8.24.0
**Architecture**: V0.9 Asset/Release two-entity model

Complete workflow for submitting, approving, and accessing geospatial data through the Platform API.

## Workflow Overview (V0.9)

```
0. Validate    POST /api/platform/submit?dry_run=true
       |
1. Submit      POST /api/platform/submit          (draft -- no version_id)
       |
2. Poll        GET /api/platform/status/{id}       (clean B2B response)
       |
3. Approve     POST /api/platform/approve          (assign version_id + clearance)
   (or Reject) POST /api/platform/reject           (reject with reason)
       |
4. Access      services block in status response    (tiles/preview/STAC URLs)
       |
5. New Version POST /api/platform/submit            (v2 draft -- no revoke needed!)
       |
6. Revoke      POST /api/platform/revoke            (optional -- removes from catalog)
```

## V0.9 Entity Model: Asset + Release

V0.9 replaces the monolithic `GeospatialAsset` with a **two-entity model**:

| Entity | Purpose | Key Fields |
|--------|---------|------------|
| **Asset** | Stable identity (never changes) | `asset_id`, `dataset_id`, `resource_id`, `data_type` |
| **Release** | Versioned artifact (many per asset) | `release_id`, `version_ordinal`, `version_id`, `approval_state` |

### Key V0.9 Changes from V0.8

| Aspect | V0.8 | V0.9 |
|--------|------|------|
| Entity model | Monolithic "asset" | Asset (identity) + Release (versioned artifact) |
| Submit body | Included `version_id`, `access_level`, `previous_version_id` | NO `version_id` at submit -- assigned at approval |
| Multi-version | Blocked (DraftBlockedError) | Coexisting approved + draft is normal |
| Revoke-first | Required before submitting v2 | NOT required -- submit v2 directly |
| Ordinal | Assigned at approval | Reserved at draft creation |
| COG path | `.../draft/...` | `.../1/...`, `.../2/...` (ordinal-based) |
| Vector table | `*_draft` | `*_ord1`, `*_ord2` (ordinal-based) |
| STAC item ID | `*-draft` | `*-ord1`, `*-ord2` (finalized to version name at approval) |
| Status response | ~140 lines flat blob | Clean B2B shape: asset/release/outputs/services/approval/versions |
| Status lookup | Only request_id | Auto-detect: request_id, job_id, release_id, asset_id |

### Release State Dimensions

| Dimension | Values | Set By |
|-----------|--------|--------|
| Processing Status | `pending`, `processing`, `completed`, `failed` | System (automatic) |
| Approval State | `pending_review`, `approved`, `rejected`, `revoked` | Reviewer |
| Clearance State | `uncleared`, `ouo`, `public` | Reviewer (during approval) |

---

## Setup & Configuration

**Environment**: Dev

### Test Data

| Type | dataset_id | resource_id | file_name |
|------|-----------|-------------|----------|
| Raster | `v09-raster-test` | `dctest` | `dctest.tif` |
| Vector | `v09-vector-test` | `cutlines` | `0403c87a-0c6c.../cutlines.gpkg` |

### Run This Cell First

In [ ]:
import requests
import json
import time
from IPython.display import Markdown, display

# =============================================================================
# ENVIRONMENT URLS (Dev)
# =============================================================================

ENVIRONMENT = "Dev"

# Platform API - submit jobs, check status, approve/reject/revoke
BASE_URL = ""

# TiTiler - raster tile serving (preview, tiles, viewer)
TITILER_URL = ""

# TiPG - vector feature serving (OGC Features API)
TIPG_URL = ""

# =============================================================================
# STORAGE CONTAINER
# =============================================================================

BRONZE_CONTAINER = "rmhazuregeobronze"

# =============================================================================
# TEST DATA (V0.9 identifiers)
# =============================================================================

# Raster test
TEST_RASTER = {
    "dataset_id": "v09-raster-test",
    "resource_id": "dctest",
    "container_name": BRONZE_CONTAINER,
    "file_name": "dctest.tif"
}

# Vector test
TEST_VECTOR = {
    "dataset_id": "v09-vector-test",
    "resource_id": "cutlines",
    "container_name": BRONZE_CONTAINER,
    "file_name": "0403c87a-0c6c-4767-a6ad-78a8026258db/Vivid_Standard_30_CO02_24Q2/cutlines.gpkg"
}

# =============================================================================
# STATE VARIABLES (populated during workflow execution)
# =============================================================================

state = {
    # Raster v1
    "raster_v1_request_id": None,
    "raster_v1_job_id": None,
    "raster_v1_release_id": None,
    "raster_asset_id": None,
    # Raster v2
    "raster_v2_request_id": None,
    "raster_v2_job_id": None,
    "raster_v2_release_id": None,
    # Vector v1
    "vector_v1_request_id": None,
    "vector_v1_job_id": None,
    "vector_v1_release_id": None,
    "vector_asset_id": None,
    # Overwrite/Reject/Revoke tests
    "ow_request_id": None,
    "rej_request_id": None,
    "rej_release_id": None,
    "rev_request_id": None,
    "rev_release_id": None,
}

# =============================================================================
# HELPER FUNCTIONS
# =============================================================================

def api_call(method, endpoint, data=None, params=None, timeout=30):
    """Make API call and return formatted response."""
    url = f"{BASE_URL}{endpoint}"
    headers = {"Content-Type": "application/json"}

    print(f"\n{'='*60}")
    print(f"{method} {endpoint}")
    print(f"{'='*60}")

    if data:
        print(f"\nRequest Body:")
        print(json.dumps(data, indent=2))

    try:
        if method == "GET":
            response = requests.get(url, params=params, timeout=timeout)
        elif method == "POST":
            response = requests.post(url, json=data, headers=headers, timeout=timeout)
        else:
            raise ValueError(f"Unsupported method: {method}")

        print(f"\nStatus: {response.status_code}")

        try:
            result = response.json()
            print(f"\nResponse:")
            print(json.dumps(result, indent=2, default=str))
            return result
        except Exception:
            print(f"\nResponse (text): {response.text[:500]}")
            return {"error": response.text, "status_code": response.status_code}

    except requests.exceptions.Timeout:
        print(f"\nRequest timed out (timeout={timeout}s)")
        return None
    except requests.exceptions.ConnectionError:
        print(f"\nCONNECTION ERROR - Check your network connection")
        print(f"   URL: {url}")
        return None
    except Exception as e:
        print(f"\nError: {e}")
        return None


def poll_status(request_id, max_polls=30, poll_interval=5):
    """Poll platform status until job completes. Returns V0.9 B2B response."""
    print(f"\n{'='*60}")
    print(f"Polling: {request_id}")
    print(f"{'='*60}")

    for i in range(max_polls):
        try:
            result = requests.get(
                f"{BASE_URL}/api/platform/status/{request_id}",
                timeout=30
            ).json()
        except Exception:
            print(f"  [{i+1}/{max_polls}] Connection error - retrying...")
            time.sleep(poll_interval)
            continue

        # V0.9 response shape
        job_status = result.get("job_status", "unknown")
        release = result.get("release") or {}
        release_id = release.get("release_id", "N/A")
        ordinal = release.get("version_ordinal", "?")
        approval = release.get("approval_state", "?")

        print(f"  [{i+1}/{max_polls}] job_status={job_status}, ordinal={ordinal}, approval={approval}")

        if job_status in ["completed", "failed"]:
            print(f"\nFinal Result:")
            print(json.dumps(result, indent=2, default=str))
            return result

        time.sleep(poll_interval)

    print(f"\nPolling timeout after {max_polls * poll_interval}s")
    return result


# =============================================================================
# CONFIGURATION SUMMARY
# =============================================================================

print("=" * 60)
print(f"PLATFORM API - V0.9 - {ENVIRONMENT} ENVIRONMENT")
print("=" * 60)
print(f"\nEndpoints:")
print(f"  Platform API:  {BASE_URL}")
print(f"  TiTiler:       {TITILER_URL}")
print(f"  TiPG:          {TIPG_URL}")
print(f"\nStorage:")
print(f"  Bronze:        {BRONZE_CONTAINER}")
print(f"\nTest Data:")
print(f"  Raster:        {TEST_RASTER['dataset_id']}/{TEST_RASTER['resource_id']} ({TEST_RASTER['file_name']})")
print(f"  Vector:        {TEST_VECTOR['dataset_id']}/{TEST_VECTOR['resource_id']}")
print("=" * 60)

---
# Part 1: Platform Health

Verify the platform is ready to accept jobs.

**Endpoint:** `GET /api/platform/health`

In [ ]:
# Platform Health Check
result = api_call("GET", "/api/platform/health")

if result:
    print(f"\n{'='*60}")
    print("PLATFORM STATUS")
    print(f"{'='*60}")
    print(f"Ready for Jobs: {result.get('ready_for_jobs', 'unknown')}")
    print(f"Queue Backlog:  {result.get('queue_backlog', 'unknown')}")
    print(f"Version:        {result.get('version', 'unknown')}")

---
# Part 2: Raster Multi-Version Lifecycle

**Goal**: Submit v1 draft -> approve -> submit v2 draft (NO revoke!) -> approve v2.

Verifies:
- Drafts have `version_id=NULL` (assigned at approval)
- COG paths use ordinals (`/1/`, `/2/`), not `draft`
- Coexisting approved v1 + draft v2
- Ordinal reserved at draft creation

Matches **V0.9_TEST.md Section A**.

## 2.1 Submit Raster Draft (v1)

Create first release. NO `version_id` in the request body -- V0.9 assigns it at approval.

**Endpoint:** `POST /api/platform/submit`

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `dataset_id` | body | Yes | Dataset identifier |
| `resource_id` | body | Yes | Resource identifier |
| `container_name` | body | Yes | Source container with raw file |
| `file_name` | body | Yes | Path to file (data type auto-detected from extension) |

**V0.9 Note**: No `version_id`, no `access_level`, no `previous_version_id`. These are V0.8 patterns.

In [ ]:
# =============================================================================
# 2.1 Submit Raster Draft (v1) -- NO version_id
# =============================================================================

submit_request = {
    "dataset_id": TEST_RASTER["dataset_id"],
    "resource_id": TEST_RASTER["resource_id"],
    "container_name": TEST_RASTER["container_name"],
    "file_name": TEST_RASTER["file_name"]
}

result = api_call("POST", "/api/platform/submit", submit_request)

if result and result.get("request_id"):
    state["raster_v1_request_id"] = result["request_id"]
    state["raster_v1_job_id"] = result.get("job_id")
    print(f"\n--- Captured ---")
    print(f"  request_id: {state['raster_v1_request_id']}")
    print(f"  job_id:     {state['raster_v1_job_id'][:16]}...")
    print(f"  job_type:   {result.get('job_type', 'N/A')}")
    print(f"  monitor:    {result.get('monitor_url', 'N/A')}")
else:
    print(f"\nSubmit failed")

## 2.2 Poll Until Completed

The V0.9 status response returns a clean B2B shape with separate `asset`, `release`, `outputs`, `services`, `approval`, and `versions` blocks.

**Endpoint:** `GET /api/platform/status/{request_id}`

In [ ]:
# =============================================================================
# 2.2 Poll Raster v1 Until Completed
# =============================================================================

if state["raster_v1_request_id"]:
    result = poll_status(state["raster_v1_request_id"], max_polls=30, poll_interval=5)

    if result and result.get("job_status") == "completed":
        # Capture release info from V0.9 response
        release = result.get("release") or {}
        asset = result.get("asset") or {}

        state["raster_v1_release_id"] = release.get("release_id")
        state["raster_asset_id"] = asset.get("asset_id")

        print(f"\n--- Captured ---")
        print(f"  release_id:       {state['raster_v1_release_id']}")
        print(f"  asset_id:         {state['raster_asset_id']}")
        print(f"  version_ordinal:  {release.get('version_ordinal')}")
        print(f"  approval_state:   {release.get('approval_state')}")
        print(f"  version_id:       {release.get('version_id')} (should be None -- draft)")

    elif result and result.get("job_status") == "failed":
        print(f"\nJob failed")
else:
    print("No request_id -- run Submit cell first")

## 2.3 Verify Outputs (Ordinal-Based COG Path)

Confirm the COG landed at `.../v09-raster-test/dctest/1/...` (ordinal), NOT `.../draft/...`.

Check the `outputs` block in the status response.

In [ ]:
# =============================================================================
# 2.3 Verify COG Path Uses Ordinal
# =============================================================================

if state["raster_v1_request_id"]:
    result = api_call("GET", f"/api/platform/status/{state['raster_v1_request_id']}")

    if result:
        outputs = result.get("outputs") or {}
        blob_path = outputs.get("blob_path", "NOT FOUND")
        stac_item_id = outputs.get("stac_item_id", "NOT FOUND")

        print(f"\n--- Verification ---")
        print(f"  blob_path:     {blob_path}")
        print(f"  stac_item_id:  {stac_item_id}")
        print(f"  container:     {outputs.get('container', 'N/A')}")

        # Verify ordinal-based path
        if "/1/" in blob_path:
            print(f"\n  PASS: COG path contains /1/ (ordinal)")
        elif "/draft/" in blob_path:
            print(f"\n  FAIL: COG path contains /draft/ -- not V0.9 compliant")
        else:
            print(f"\n  CHECK: Unexpected path pattern")
else:
    print("No request_id -- run previous cells first")

## 2.4 Approve v1

V0.9 approval uses `release_id` (preferred) and assigns `version_id` + `clearance_level`.

**Endpoint:** `POST /api/platform/approve`

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `release_id` | body | Yes (preferred) | Target release to approve |
| `reviewer` | body | Yes | Email of approver |
| `clearance_level` | body | Yes | `ouo` or `public` |
| `version_id` | body | Yes | Human-readable version label (e.g., `v1`) |

**Expected**: STAC item materialized with versioned ID (e.g., `v09-raster-test-dctest-v1`).

In [ ]:
# =============================================================================
# 2.4 Approve Raster v1
# =============================================================================

if state["raster_v1_release_id"]:
    approve_request = {
        "release_id": state["raster_v1_release_id"],
        "reviewer": "qa-tester@example.com",
        "clearance_level": "ouo",
        "version_id": "v1"
    }

    result = api_call("POST", "/api/platform/approve", approve_request)

    if result and result.get("success"):
        print(f"\n--- Result ---")
        print(f"  approval_state:  {result.get('approval_state')}")
        print(f"  clearance_state: {result.get('clearance_state')}")
        print(f"  stac_item_id:    {result.get('stac_item_id')}")
        print(f"  stac_updated:    {result.get('stac_updated')}")
        print(f"  release_id:      {result.get('release_id')}")
    else:
        print(f"\nApproval failed")
else:
    print("No release_id -- run Poll cell first (job must be completed)")

## 2.5 Submit v2 Draft (Coexisting Versions -- NO Revoke Needed!)

**V0.9 Flagship Feature**: Submit a new version while v1 is still approved.
Should NOT return 409 DraftBlockedError.

Same identifiers as v1 -- the system auto-creates a new release with `version_ordinal=2`.

In [ ]:
# =============================================================================
# 2.5 Submit Raster v2 (NO revoke -- coexisting versions)
# =============================================================================

submit_v2_request = {
    "dataset_id": TEST_RASTER["dataset_id"],
    "resource_id": TEST_RASTER["resource_id"],
    "container_name": TEST_RASTER["container_name"],
    "file_name": TEST_RASTER["file_name"]
}

result = api_call("POST", "/api/platform/submit", submit_v2_request)

if result and result.get("request_id"):
    state["raster_v2_request_id"] = result["request_id"]
    state["raster_v2_job_id"] = result.get("job_id")
    print(f"\n--- Captured ---")
    print(f"  request_id: {state['raster_v2_request_id']}")
    print(f"  job_id:     {state['raster_v2_job_id'][:16]}...")
    print(f"  HTTP 202 (NOT 409) -- coexisting versions work!")
elif result and result.get("status_code") == 409:
    print(f"\nFAIL: Got 409 DraftBlockedError -- V0.9 should allow coexisting versions")
else:
    print(f"\nSubmit failed")

## 2.6 Poll v2, Verify Ordinal=2

The new release should have `version_ordinal=2` (auto-incremented from v1's ordinal=1).

In [ ]:
# =============================================================================
# 2.6 Poll Raster v2 + Verify Ordinal
# =============================================================================

if state["raster_v2_request_id"]:
    result = poll_status(state["raster_v2_request_id"], max_polls=30, poll_interval=5)

    if result and result.get("job_status") == "completed":
        release = result.get("release") or {}
        outputs = result.get("outputs") or {}

        state["raster_v2_release_id"] = release.get("release_id")

        print(f"\n--- Captured ---")
        print(f"  release_id:       {state['raster_v2_release_id']}")
        print(f"  version_ordinal:  {release.get('version_ordinal')} (should be 2)")
        print(f"  approval_state:   {release.get('approval_state')} (should be pending_review)")
        print(f"  blob_path:        {outputs.get('blob_path', 'N/A')}")

        # Verify ordinal isolation
        blob_path = outputs.get("blob_path", "")
        if "/2/" in blob_path:
            print(f"\n  PASS: v2 COG path contains /2/ (ordinal isolation)")
        else:
            print(f"\n  CHECK: Expected /2/ in path")
else:
    print("No v2 request_id -- run Submit v2 cell first")

## 2.7 Approve v2

Approve the second release. Both v1 and v2 will be approved in the catalog.

In [ ]:
# =============================================================================
# 2.7 Approve Raster v2
# =============================================================================

if state["raster_v2_release_id"]:
    approve_v2_request = {
        "release_id": state["raster_v2_release_id"],
        "reviewer": "qa-tester@example.com",
        "clearance_level": "ouo",
        "version_id": "v2"
    }

    result = api_call("POST", "/api/platform/approve", approve_v2_request)

    if result and result.get("success"):
        print(f"\n--- Result ---")
        print(f"  approval_state:  {result.get('approval_state')}")
        print(f"  stac_item_id:    {result.get('stac_item_id')}")
        print(f"  stac_updated:    {result.get('stac_updated')}")
    else:
        print(f"\nApproval failed")
else:
    print("No v2 release_id -- run Poll v2 cell first")

---
# Part 3: Status Lookups (Auto-Detect)

V0.9 introduced auto-detect ID lookups: `GET /api/platform/status/{id}` accepts any of:
- `request_id` (direct api_requests lookup)
- `job_id` (api_requests.job_id match)
- `release_id` (release -> job_id -> api_requests)
- `asset_id` (asset -> latest release -> job_id -> api_requests)

Resolution order: request_id -> job_id -> release_id -> asset_id

All return the same clean B2B response shape.

In [ ]:
# =============================================================================
# 3.1 Lookup by request_id
# =============================================================================

if state["raster_v1_request_id"]:
    print("--- Lookup by request_id ---")
    result = api_call("GET", f"/api/platform/status/{state['raster_v1_request_id']}")

    if result and result.get("success"):
        print(f"\n  PASS: request_id lookup returned success")
        print(f"  job_status:     {result.get('job_status')}")
        print(f"  asset_id:       {result.get('asset', {}).get('asset_id', 'N/A')}")
        print(f"  release count:  {result.get('asset', {}).get('release_count', 'N/A')}")
else:
    print("No request_id available")

In [ ]:
# =============================================================================
# 3.2 Lookup by asset_id (auto-detect)
# =============================================================================

if state["raster_asset_id"]:
    print("--- Lookup by asset_id ---")
    result = api_call("GET", f"/api/platform/status/{state['raster_asset_id']}")

    if result and result.get("success"):
        print(f"\n  PASS: asset_id auto-detect resolved")
        print(f"  Shows latest release for this asset")
        release = result.get("release") or {}
        print(f"  release_id:      {release.get('release_id', 'N/A')}")
        print(f"  version_ordinal: {release.get('version_ordinal', 'N/A')}")
    else:
        print(f"\n  FAIL: asset_id lookup returned error")
else:
    print("No asset_id available -- run Part 2 first")

In [ ]:
# =============================================================================
# 3.3 Lookup by release_id (auto-detect)
# =============================================================================

if state["raster_v1_release_id"]:
    print("--- Lookup by release_id ---")
    result = api_call("GET", f"/api/platform/status/{state['raster_v1_release_id']}")

    if result and result.get("success"):
        print(f"\n  PASS: release_id auto-detect resolved")
        release = result.get("release") or {}
        print(f"  release_id:      {release.get('release_id', 'N/A')}")
        print(f"  version_id:      {release.get('version_id', 'N/A')}")
        print(f"  approval_state:  {release.get('approval_state', 'N/A')}")
    else:
        print(f"\n  FAIL: release_id lookup returned error")
else:
    print("No release_id available -- run Part 2 first")

In [ ]:
# =============================================================================
# 3.4 Lookup by dataset_id + resource_id (query params)
# =============================================================================

print("--- Lookup by dataset_id + resource_id ---")
result = api_call(
    "GET",
    "/api/platform/status",
    params={
        "dataset_id": TEST_RASTER["dataset_id"],
        "resource_id": TEST_RASTER["resource_id"]
    }
)

if result and result.get("success"):
    print(f"\n  PASS: dataset_id+resource_id lookup resolved")
    versions = result.get("versions") or []
    print(f"  versions found: {len(versions)}")
    for v in versions:
        print(f"    ordinal={v.get('version_ordinal')}, "
              f"version_id={v.get('version_id')}, "
              f"approval={v.get('approval_state')}, "
              f"is_latest={v.get('is_latest')}")

## 3.5 With `?detail=full`

Appends operational detail (job_result, task_summary, admin URLs) to the clean B2B response.

**Endpoint:** `GET /api/platform/status/{id}?detail=full`

In [ ]:
# =============================================================================
# 3.5 Status with ?detail=full
# =============================================================================

if state["raster_v1_request_id"]:
    print("--- Status with ?detail=full ---")
    result = api_call("GET", f"/api/platform/status/{state['raster_v1_request_id']}?detail=full")

    if result:
        # Check for detail block
        detail = result.get("detail")
        if detail:
            print(f"\n--- Detail Block ---")
            print(f"  job_id:      {detail.get('job_id', 'N/A')[:24]}...")
            print(f"  job_type:    {detail.get('job_type', 'N/A')}")
            print(f"  job_stage:   {detail.get('job_stage', 'N/A')}")
            print(f"  created_at:  {detail.get('created_at', 'N/A')}")

            task_summary = detail.get("task_summary") or {}
            print(f"\n  Task Summary:")
            print(f"    total:     {task_summary.get('total', 'N/A')}")
            print(f"    completed: {task_summary.get('completed', 'N/A')}")
            print(f"    failed:    {task_summary.get('failed', 'N/A')}")

            urls = detail.get("urls") or {}
            print(f"\n  Admin URLs:")
            print(f"    job_status: {urls.get('job_status', 'N/A')}")
            print(f"    job_tasks:  {urls.get('job_tasks', 'N/A')}")

            print(f"\n  PASS: ?detail=full appends operational detail")
        else:
            print(f"\n  FAIL: No 'detail' block in response")
else:
    print("No request_id available")

---
# Part 4: Vector Workflow

**Goal**: Submit vector draft -> poll -> approve.
Verify ordinal-based table naming (`*_ord1` not `*_draft`) and ordinal-based STAC IDs.

Matches **V0.9_TEST.md Section B**.

## 4.1 Submit Vector Draft

Same submit pattern as raster -- no `version_id` in request body.

In [ ]:
# =============================================================================
# 4.1 Submit Vector Draft
# =============================================================================

submit_vector = {
    "dataset_id": TEST_VECTOR["dataset_id"],
    "resource_id": TEST_VECTOR["resource_id"],
    "container_name": TEST_VECTOR["container_name"],
    "file_name": TEST_VECTOR["file_name"]
}

result = api_call("POST", "/api/platform/submit", submit_vector)

if result and result.get("request_id"):
    state["vector_v1_request_id"] = result["request_id"]
    state["vector_v1_job_id"] = result.get("job_id")
    print(f"\n--- Captured ---")
    print(f"  request_id: {state['vector_v1_request_id']}")
    print(f"  job_id:     {state['vector_v1_job_id'][:16]}...")
    print(f"  job_type:   {result.get('job_type', 'N/A')}")
else:
    print(f"\nSubmit failed")

## 4.2 Poll + Verify Ordinal Table Name

Expected table name: `v09_vector_test_cutlines_ord1` (NOT `*_draft`).

In [ ]:
# =============================================================================
# 4.2 Poll Vector + Verify Ordinal Table Name
# =============================================================================

if state["vector_v1_request_id"]:
    result = poll_status(state["vector_v1_request_id"], max_polls=30, poll_interval=5)

    if result and result.get("job_status") == "completed":
        release = result.get("release") or {}
        asset = result.get("asset") or {}
        outputs = result.get("outputs") or {}

        state["vector_v1_release_id"] = release.get("release_id")
        state["vector_asset_id"] = asset.get("asset_id")

        table_name = outputs.get("table_name", "NOT FOUND")

        print(f"\n--- Captured ---")
        print(f"  release_id:       {state['vector_v1_release_id']}")
        print(f"  asset_id:         {state['vector_asset_id']}")
        print(f"  version_ordinal:  {release.get('version_ordinal')}")
        print(f"  table_name:       {table_name}")
        print(f"  schema:           {outputs.get('schema', 'N/A')}")
        print(f"  stac_item_id:     {outputs.get('stac_item_id', 'N/A')}")

        # Verify ordinal-based table name
        if "_ord1" in table_name:
            print(f"\n  PASS: Table name uses _ord1 (ordinal)")
        elif "_draft" in table_name:
            print(f"\n  FAIL: Table name uses _draft -- not V0.9 compliant")
        else:
            print(f"\n  CHECK: Unexpected table name pattern")
else:
    print("No vector request_id -- run Submit cell first")

## 4.3 Approve Vector v1

Same approval pattern as raster -- `release_id` + `version_id` + `clearance_level` + `reviewer`.

In [ ]:
# =============================================================================
# 4.3 Approve Vector v1
# =============================================================================

if state["vector_v1_release_id"]:
    approve_vector = {
        "release_id": state["vector_v1_release_id"],
        "reviewer": "qa-tester@example.com",
        "clearance_level": "ouo",
        "version_id": "v1"
    }

    result = api_call("POST", "/api/platform/approve", approve_vector)

    if result and result.get("success"):
        print(f"\n--- Result ---")
        print(f"  approval_state:  {result.get('approval_state')}")
        print(f"  stac_item_id:    {result.get('stac_item_id')}")
        print(f"  stac_updated:    {result.get('stac_updated')}")
    else:
        print(f"\nApproval failed")
else:
    print("No vector release_id -- run Poll cell first")

---
# Part 5: Overwrite & Reject/Resubmit

**Goal**: Test draft overwrite (`processing_options.overwrite=true`), reject, and resubmit workflows.

Matches **V0.9_TEST.md Sections D and E**.

## 5.1 Submit Fresh Draft (for overwrite test)

Create a new draft to test the overwrite flow.

In [ ]:
# =============================================================================
# 5.1 Submit Fresh Draft
# =============================================================================

submit_ow = {
    "dataset_id": "v09-overwrite-test",
    "resource_id": "ow-raster",
    "container_name": BRONZE_CONTAINER,
    "file_name": "dctest.tif"
}

result = api_call("POST", "/api/platform/submit", submit_ow)

if result and result.get("request_id"):
    state["ow_request_id"] = result["request_id"]
    print(f"\n--- Captured ---")
    print(f"  request_id: {state['ow_request_id']}")
    print(f"  job_id:     {result.get('job_id', 'N/A')[:16]}...")
    print(f"\n  Waiting for completion before overwrite...")

    # Poll to completion
    poll_result = poll_status(state["ow_request_id"], max_polls=30, poll_interval=5)
    if poll_result and poll_result.get("job_status") == "completed":
        print(f"\n  Draft completed -- ready for overwrite test")
else:
    print(f"\nSubmit failed")

## 5.2 Overwrite Draft (processing_options.overwrite=true)

Reprocess the same draft with `overwrite=true`. This creates a new job but reuses the existing release.

**Expected**: Same `release_id`, new `job_id`, revision incremented.

In [ ]:
# =============================================================================
# 5.2 Overwrite Draft
# =============================================================================

submit_overwrite = {
    "dataset_id": "v09-overwrite-test",
    "resource_id": "ow-raster",
    "container_name": BRONZE_CONTAINER,
    "file_name": "dctest.tif",
    "processing_options": {"overwrite": True}
}

result = api_call("POST", "/api/platform/submit", submit_overwrite)

if result and result.get("request_id"):
    print(f"\n--- Result ---")
    print(f"  request_id: {result.get('request_id')}")
    print(f"  job_id:     {result.get('job_id', 'N/A')[:16]}... (should be different from 5.1)")
    print(f"  Overwrite accepted -- new job created for same release")
elif result and "idempotent" in str(result).lower():
    print(f"\n  Without overwrite=true, this would be idempotent (existing draft returned)")
else:
    print(f"\nOverwrite failed")

## 5.3 Reject a Release

Submit a fresh draft, then reject it.

**Endpoint:** `POST /api/platform/reject`

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `release_id` | body | Yes | Release to reject |
| `reviewer` | body | Yes | Email of reviewer |
| `reason` | body | Yes | Reason for rejection (audit trail) |

In [ ]:
# =============================================================================
# 5.3 Submit + Reject
# =============================================================================

# Step 1: Submit a draft for rejection test
submit_rej = {
    "dataset_id": "v09-reject-test",
    "resource_id": "rej-raster",
    "container_name": BRONZE_CONTAINER,
    "file_name": "dctest.tif"
}

result = api_call("POST", "/api/platform/submit", submit_rej)

if result and result.get("request_id"):
    state["rej_request_id"] = result["request_id"]
    print(f"\n  Waiting for completion...")

    poll_result = poll_status(state["rej_request_id"], max_polls=30, poll_interval=5)

    if poll_result and poll_result.get("job_status") == "completed":
        release = poll_result.get("release") or {}
        state["rej_release_id"] = release.get("release_id")

        # Step 2: Reject
        reject_request = {
            "release_id": state["rej_release_id"],
            "reviewer": "qa-tester@example.com",
            "reason": "V0.9 notebook test rejection"
        }

        reject_result = api_call("POST", "/api/platform/reject", reject_request)

        if reject_result and reject_result.get("approval_state") == "rejected":
            print(f"\n  PASS: Release rejected")
            print(f"  release_id:     {reject_result.get('release_id')}")
            print(f"  approval_state: {reject_result.get('approval_state')}")
        else:
            print(f"\n  Rejection failed")
else:
    print(f"\nSubmit failed")

## 5.4 Resubmit via Platform Resubmit

After rejection, resubmit using the `request_id`. Cleans up old artifacts and creates a new job.

**Endpoint:** `POST /api/platform/resubmit`

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `request_id` | body | Yes | Original request ID |

In [ ]:
# =============================================================================
# 5.4 Resubmit After Rejection
# =============================================================================

if state["rej_request_id"]:
    resubmit_request = {
        "request_id": state["rej_request_id"]
    }

    result = api_call("POST", "/api/platform/resubmit", resubmit_request)

    if result and result.get("success"):
        print(f"\n--- Result ---")
        print(f"  original_job_id: {result.get('original_job_id', 'N/A')[:16]}...")
        print(f"  new_job_id:      {result.get('new_job_id', 'N/A')[:16]}...")
        print(f"  cleanup_summary: {result.get('cleanup_summary', {})}")
        print(f"  monitor_url:     {result.get('monitor_url', 'N/A')}")

        # Poll the resubmitted job
        print(f"\n  Polling resubmitted job...")
        poll_result = poll_status(state["rej_request_id"], max_polls=30, poll_interval=5)
        if poll_result and poll_result.get("job_status") == "completed":
            print(f"\n  PASS: Resubmitted job completed")
    else:
        print(f"\n  Resubmit failed")
else:
    print("No rej_request_id -- run Reject cell first")

---
# Part 6: Revoke

**Goal**: Approve a release, then revoke it. Verify STAC item is deleted.

Matches **V0.9_TEST.md Section F**.

**Endpoint:** `POST /api/platform/revoke`

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `release_id` | body | Yes | Release to revoke |
| `revoker` | body | Yes | Email of person revoking |
| `reason` | body | Yes | Reason for revocation (audit trail) |

In [ ]:
# =============================================================================
# 6.1 Submit + Approve + Revoke
# =============================================================================

# Step 1: Submit
submit_rev = {
    "dataset_id": "v09-revoke-test",
    "resource_id": "rev-raster",
    "container_name": BRONZE_CONTAINER,
    "file_name": "dctest.tif"
}

result = api_call("POST", "/api/platform/submit", submit_rev)

if result and result.get("request_id"):
    state["rev_request_id"] = result["request_id"]
    print(f"\n  Waiting for completion...")

    poll_result = poll_status(state["rev_request_id"], max_polls=30, poll_interval=5)

    if poll_result and poll_result.get("job_status") == "completed":
        release = poll_result.get("release") or {}
        state["rev_release_id"] = release.get("release_id")

        # Step 2: Approve
        approve_rev = {
            "release_id": state["rev_release_id"],
            "reviewer": "qa-tester@example.com",
            "clearance_level": "ouo",
            "version_id": "v1"
        }

        approve_result = api_call("POST", "/api/platform/approve", approve_rev)

        if approve_result and approve_result.get("success"):
            stac_item_id = approve_result.get("stac_item_id")
            print(f"\n  Approved. STAC item: {stac_item_id}")

            # Step 3: Revoke
            revoke_request = {
                "release_id": state["rev_release_id"],
                "revoker": "qa-tester@example.com",
                "reason": "V0.9 notebook revoke test"
            }

            revoke_result = api_call("POST", "/api/platform/revoke", revoke_request)

            if revoke_result and revoke_result.get("approval_state") == "revoked":
                print(f"\n  PASS: Release revoked")
                print(f"  approval_state: {revoke_result.get('approval_state')}")
                print(f"  stac_updated:   {revoke_result.get('stac_updated')}")
            else:
                print(f"\n  Revoke failed")
        else:
            print(f"\n  Approve step failed")
else:
    print(f"\nSubmit failed")

## 6.2 Verify STAC Item Deleted

After revocation, the STAC item should no longer exist in the catalog.

In [ ]:
# =============================================================================
# 6.2 Verify STAC Item Deleted After Revoke
# =============================================================================

# The STAC item ID pattern: {dataset_id}-{resource_id}-{version_id}
stac_item_id = "v09-revoke-test-rev-raster-v1"
collection_id = "v09-revoke-test"

result = api_call("GET", f"/api/platform/catalog/item/{collection_id}/{stac_item_id}")

if result and result.get("status_code") == 404:
    print(f"\n  PASS: STAC item deleted (HTTP 404)")
elif result and result.get("type") == "Feature":
    print(f"\n  FAIL: STAC item still exists after revoke")
else:
    print(f"\n  Result: {result}")

---
# Part 7: Catalog Discovery

Query published assets and STAC items through the Platform API.

## Catalog Endpoints

| Endpoint | Purpose |
|----------|---------|
| `GET /api/platform/catalog/assets` | List all assets with filters |
| `GET /api/platform/catalog/lookup` | Lookup by DDH IDs |
| `GET /api/platform/catalog/asset/{asset_id}` | Get asset with service URLs |
| `GET /api/platform/catalog/item/{col}/{item}` | Get STAC item |
| `GET /api/platform/catalog/dataset/{id}` | List items for a dataset |

In [ ]:
# =============================================================================
# 7.1 List Assets
# =============================================================================

result = api_call("GET", "/api/platform/catalog/assets", params={"limit": 10})

if result and result.get("assets"):
    assets = result["assets"]
    print(f"\nFound {len(assets)} assets:")
    for asset in assets[:5]:
        print(f"  - {asset.get('dataset_id')}/{asset.get('resource_id')}")
        print(f"    data_type: {asset.get('data_type')}, "
              f"approval: {asset.get('approval_state')}, "
              f"clearance: {asset.get('clearance_state')}")

In [ ]:
# =============================================================================
# 7.2 Lookup by DDH Identifiers
# =============================================================================

params = {
    "dataset_id": TEST_RASTER["dataset_id"],
    "resource_id": TEST_RASTER["resource_id"]
}

result = api_call("GET", "/api/platform/catalog/lookup", params=params)

if result and result.get("found"):
    print(f"\n  Asset found")
    print(f"  stac_item_id:  {result.get('stac_item_id', 'N/A')}")
    print(f"  collection_id: {result.get('stac_collection_id', 'N/A')}")
    print(f"  approval:      {result.get('approval_state', 'N/A')}")
else:
    print(f"\n  Asset not found")

In [ ]:
# =============================================================================
# 7.3 Get Asset Detail (with service URLs)
# =============================================================================

if state["raster_asset_id"]:
    result = api_call("GET", f"/api/platform/catalog/asset/{state['raster_asset_id']}")

    if result and result.get("found"):
        print(f"\n--- Asset Detail ---")
        print(f"  asset_id:    {result.get('asset_id')}")
        print(f"  data_type:   {result.get('data_type')}")

        status = result.get("status") or {}
        print(f"  processing:  {status.get('processing')}")
        print(f"  approval:    {status.get('approval')}")
        print(f"  clearance:   {status.get('clearance')}")

        ddh = result.get("ddh_refs") or {}
        print(f"  version_id:  {ddh.get('version_id')}")

        lineage = result.get("lineage") or {}
        print(f"  ordinal:     {lineage.get('version_ordinal')}")
        print(f"  is_latest:   {lineage.get('is_latest')}")

        # Service URLs
        raster = result.get("raster") or {}
        tiles = raster.get("tiles") or {}
        if tiles:
            print(f"\n  Service URLs:")
            print(f"    preview:    {tiles.get('preview', 'N/A')[:60]}...")
            print(f"    tiles:      {tiles.get('xyz', 'N/A')[:60]}...")
            print(f"    viewer:     {tiles.get('viewer', 'N/A')[:60]}...")
else:
    print("No asset_id -- run Part 2 first")

In [ ]:
# =============================================================================
# 7.4 Get STAC Item
# =============================================================================

# Raster v2 approved item
COLLECTION_ID = "v09-raster-test"
ITEM_ID = "v09-raster-test-dctest-v2"

result = api_call("GET", f"/api/platform/catalog/item/{COLLECTION_ID}/{ITEM_ID}")

if result and result.get("type") == "Feature":
    props = result.get("properties", {})
    print(f"\n  STAC Item retrieved")
    print(f"  id:         {result.get('id')}")
    print(f"  collection: {result.get('collection')}")
    print(f"  bbox:       {result.get('bbox', 'N/A')}")
    assets = result.get("assets", {})
    print(f"  assets:     {list(assets.keys())}")
elif result:
    print(f"\n  Item not found or error")

In [ ]:
# =============================================================================
# 7.5 List Items for Dataset
# =============================================================================

DATASET_ID = TEST_RASTER["dataset_id"]

result = api_call(
    "GET",
    f"/api/platform/catalog/dataset/{DATASET_ID}",
    params={"limit": 50}
)

if result and result.get("items"):
    items = result["items"]
    print(f"\nFound {result.get('count', len(items))} items for '{DATASET_ID}':")
    for item in items[:10]:
        print(f"  - {item.get('item_id')}")
        print(f"    resource: {item.get('resource_id')}, version: {item.get('version_id')}")
elif result and result.get("count") == 0:
    print(f"\nNo items found for '{DATASET_ID}'")

---
# Part 8: Approvals Management

Query and inspect releases by approval state.

## Endpoints

| Endpoint | Purpose |
|----------|---------|
| `GET /api/platform/approvals?status=X` | List releases by approval state |
| `GET /api/platform/approvals/{release_id}` | Get single release detail |

In [ ]:
# =============================================================================
# 8.1 List Releases (approved)
# =============================================================================

result = api_call("GET", "/api/platform/approvals", params={"status": "approved"})

if result and result.get("releases"):
    releases = result["releases"]
    print(f"\nApproved releases: {result.get('count', len(releases))}")
    print(f"Status counts: {result.get('status_counts', {})}")

    for r in releases[:10]:
        print(f"\n  release_id:  {r.get('release_id', 'N/A')[:20]}...")
        print(f"  version_id:  {r.get('version_id')}")
        print(f"  ordinal:     {r.get('version_ordinal')}")
        print(f"  is_latest:   {r.get('is_latest')}")
        print(f"  stac_item:   {r.get('stac_item_id')}")
        print(f"  reviewer:    {r.get('reviewer')}")

In [ ]:
# =============================================================================
# 8.2 Get Release Detail
# =============================================================================

if state["raster_v1_release_id"]:
    result = api_call("GET", f"/api/platform/approvals/{state['raster_v1_release_id']}")

    if result and result.get("release"):
        r = result["release"]
        print(f"\n--- Release Detail ---")
        print(f"  release_id:      {r.get('release_id')}")
        print(f"  asset_id:        {r.get('asset_id')}")
        print(f"  version_id:      {r.get('version_id')}")
        print(f"  version_ordinal: {r.get('version_ordinal')}")
        print(f"  revision:        {r.get('revision')}")
        print(f"  is_latest:       {r.get('is_latest')}")
        print(f"  is_served:       {r.get('is_served')}")
        print(f"  blob_path:       {r.get('blob_path')}")
        print(f"  stac_item_id:    {r.get('stac_item_id')}")
        print(f"  approval_state:  {r.get('approval_state')}")
        print(f"  clearance_state: {r.get('clearance_state')}")
        print(f"  reviewer:        {r.get('reviewer')}")
        print(f"  reviewed_at:     {r.get('reviewed_at')}")
    else:
        print(f"\n  Release not found")
else:
    print("No release_id -- run Part 2 first")

---

## API Quick Reference

### Workflow Endpoints

| Endpoint | Method | Purpose |
|----------|--------|---------|
| `/api/platform/health` | GET | Platform health check |
| `/api/platform/submit` | POST | Submit draft (no version_id) |
| `/api/platform/submit?dry_run=true` | POST | Validate without submitting |
| `/api/platform/status/{id}` | GET | Status (auto-detect ID type) |
| `/api/platform/status/{id}?detail=full` | GET | Status + operational detail |
| `/api/platform/approve` | POST | Approve release (assign version_id) |
| `/api/platform/reject` | POST | Reject release with reason |
| `/api/platform/revoke` | POST | Revoke approved release |
| `/api/platform/resubmit` | POST | Resubmit after rejection |

### Catalog Endpoints

| Endpoint | Method | Purpose |
|----------|--------|---------|
| `/api/platform/catalog/assets` | GET | List assets with filters |
| `/api/platform/catalog/lookup` | GET | Lookup by DDH IDs |
| `/api/platform/catalog/asset/{id}` | GET | Asset detail with service URLs |
| `/api/platform/catalog/item/{col}/{item}` | GET | STAC item GeoJSON |
| `/api/platform/catalog/dataset/{id}` | GET | Items for a dataset |
| `/api/platform/approvals` | GET | List releases by status |
| `/api/platform/approvals/{release_id}` | GET | Single release detail |

### V0.9 Request Bodies

**Submit** (no version_id, no access_level, no previous_version_id):
```json
{
    "dataset_id": "...",
    "resource_id": "...",
    "container_name": "...",
    "file_name": "..."
}
```

**Approve** (release_id preferred):
```json
{
    "release_id": "...",
    "reviewer": "...",
    "clearance_level": "ouo",
    "version_id": "v1"
}
```

**Reject**:
```json
{
    "release_id": "...",
    "reviewer": "...",
    "reason": "..."
}
```

**Revoke**:
```json
{
    "release_id": "...",
    "revoker": "...",
    "reason": "..."
}
```

**Resubmit**:
```json
{
    "request_id": "..."
}
```

**Overwrite**:
```json
{
    "dataset_id": "...",
    "resource_id": "...",
    "container_name": "...",
    "file_name": "...",
    "processing_options": {"overwrite": true}
}
```

### V0.9 B2B Response Shape

All `/api/platform/status/{id}` lookups return:

| Block | Contents | When Present |
|-------|----------|--------------|
| `asset` | Stable identity: asset_id, dataset_id, resource_id, data_type, release_count | Always |
| `release` | Lifecycle: release_id, version_id, ordinal, approval/clearance/processing | Always |
| `job_status` | Single string: pending/processing/completed/failed | Always |
| `outputs` | Artifacts: blob_path or table_name, stac_item_id, container/schema | When completed |
| `services` | URLs: preview/tiles/viewer (raster) or collection/items (vector) | When completed |
| `approval` | Workflow: approve_url, viewer_url, embed_url | When pending_review + completed |
| `versions` | All releases for this asset (newest first) | Always |
| `detail` | Operational: job_result, task_summary, admin URLs | Only with `?detail=full` |

### Release State Model

| Dimension | Values | Set By |
|-----------|--------|--------|
| Processing Status | `pending`, `processing`, `completed`, `failed` | System (automatic) |
| Approval State | `pending_review`, `approved`, `rejected`, `revoked` | Reviewer (approve/reject/revoke) |
| Clearance State | `uncleared`, `ouo`, `public` | Reviewer (during approval) |

---

*Platform API V0.9 -- Version 0.8.24.0*
*Last Updated: 23 FEB 2026*